# DeepCAD-RT testing pipeline 
<img src="https://github.com/STAR-811/DeepCAD-RT-old/blob/main/images/logo-new.png?raw=true" width = "650" height = "180" align=right />

This file will demonstrate the basic pipeline for denoising specified data using pre-trained DeepCAD-RT models. A TIFF file will be downloaded automatically to be the example data. More information about the method and relevant results can be found in the companion paper：

**Real-time denoising of fluorescence time-lapse imaging enables high-sensitivity observations of biological dynamics beyond the shot-noise limit. bioRxiv (2022).**

In [1]:
from deepcad.test_collection import testing_class
from deepcad.movie_display import display
from deepcad.utils import get_first_filename, download_demo
import h5py
import os

## Select the file(s) to be processed (download if not exist)
The `download_demo` function will download demo files (example data and a pre-trained model) and return the full paths of them. The data will be saved in `/datasets` and the model will be saved in `/pth`. If you want to use your own data for testing, please create a new folder in `/datasets` and copy your data into it. Then, just change `datasets_path` into the name of your dataset folder. All TIFF files inside `datasets_path` will be tested. If you want to use your own model for testing, please change `denoise_model` into the folder containing your model(s). All models inside `denoise_model` will be tested. 

In [13]:
metainfo = h5py.File(os.getcwd()+'/00_Metafile/metafile.h5', 'r')

recordingDate = metainfo['sessionInfo/recordingDate'][()].decode()
mouseID = metainfo['sessionInfo/mouseID'][()].decode()
behaviorInfo = metainfo['sessionInfo/behaviorInfo'][()].decode()
recTarget = metainfo['sessionInfo/recTarget'][()].decode()
sensor = metainfo['sessionInfo/sensor'][()].decode()
fs = metainfo['sessionInfo/fs'][()]
resolution = metainfo['sessionInfo/resolution'][()]
analysisDate = metainfo['sessionInfo/analysisDate'][()].decode()


analysisDir = metainfo['sessionInfo/analysisDir'][()].decode()+'/01_ROI_detection_DeepCAD_all' 
datasets_path = analysisDir+'/suite2p/plane0/reg_tif'
metainfo.close()

print('recording data:', recordingDate)
print('analysis data:', analysisDate)
print('behaviorInfo:', behaviorInfo)
print('sensor:', sensor)
print('recTarget:', recTarget)
print('fs:', fs)
print('resolution:', resolution)

print('analysisDir:', analysisDir)
print('dataDir_for_DeepCAD:', datasets_path)

recording data: 
analysis data: 2023-05-26 16:03:05.694461
behaviorInfo: 
sensor: GCaMP7.09
recTarget: Soma
fs: 7.65
resolution: 2048
analysisDir: /mnt/ssd1/ysaito/suite2p-pipeline-main/02_analysis/i166-m2_Exp01/01_ROI_detection_DeepCAD_all
dataDir_for_DeepCAD: /mnt/ssd1/ysaito/suite2p-pipeline-main/02_analysis/i166-m2_Exp01/01_ROI_detection_DeepCAD_all/suite2p/plane0/reg_tif


In [15]:
denoise_model =  'data_all_202305261842'

## Set the parameters for testing
Default setting shows the parameters suitable for the demo file. You can change these parameters according to your data and device. To visualize the testing process, you can set the flags `visualize_images_per_epoch` and `save_test_images_per_epoch` according to your demands.

In [16]:
test_datasize = 8001                   # the number of frames to be tested
GPU = '0'                             # the index of GPU you will use for computation (e.g. '0', '0,1', '0,1,2')
patch_xy = 150                        # the width and height of 3D patches
patch_t = 150                         # the time dimension (frames) of 3D patches
overlap_factor = 0.6                  # the overlap factor between two adjacent patches
# Since the receptive field of 3D-Unet is ~90, seamless stitching requires an overlap (patch_xyt*overlap_factor）of at least 90 pixels.
num_workers = 0                       # if you use Windows system, set this to 0.

# Setup some parameters for result visualization during the test (optional)
visualize_images_per_epoch = False    # whether to display inference performance after each epoch
save_test_images_per_epoch = True     # whether to save inference image after each epoch in pth path

##  Show the input low-SNR data  (optional)
Play an input video (optional). This will load the video into memory and it is not an indispensable step. OpenCV library was used for display.

In [17]:
# display_images = True

# if display_images:
#     display_filename = get_first_filename(datasets_path)
#     print('\033[1;31mDisplaying the first raw file -----> \033[0m')
#     print(display_filename)
#     display_length = 300  # the frames number of the noise movie
#     # normalize the image and display
#     display(display_filename, display_length=display_length, norm_min_percent=1, norm_max_percent=98)

## Create a testing object
This will creat a testing object by passing all parameters as a dictionary. Parameters not specified in the dictionary will use their default values.

In [18]:
test_dict = {
    # dataset dependent parameters
    'patch_x': patch_xy,                 # the width of 3D patches
    'patch_y': patch_xy,                 # the height of 3D patches
    'patch_t': patch_t,                  # the time dimension (frames) of 3D patches
    'overlap_factor':overlap_factor,     # overlap factor, 
    'scale_factor': 1,                   # the factor for image intensity scaling
    'test_datasize': test_datasize,      # the number of frames to be tested
    'datasets_path': datasets_path,      # folder containing all files to be tested
    'pth_dir': './pth',                  # pth file root path
    'denoise_model' : denoise_model,     # A folder containing all models to be tested
    'output_dir' : datasets_path,          # result file root path
    # network related parameters
    'fmap': 16,                          # number of feature maps
    'GPU': GPU,                          # GPU index
    'num_workers': num_workers,          # if you use Windows system, set this to 0.
    'visualize_images_per_epoch': visualize_images_per_epoch,  # whether to display inference performance after each epoch
    'save_test_images_per_epoch': save_test_images_per_epoch   # whether to save inference image after each epoch in pth path
}

tc = testing_class(test_dict)

Testing parameters -----> 
{'overlap_factor': 0.6, 'datasets_path': '/mnt/ssd1/ysaito/suite2p-pipeline-main/02_analysis/i166-m2_Exp01/01_ROI_detection_DeepCAD_all/suite2p/plane0/reg_tif', 'fmap': 16, 'output_dir': '/mnt/ssd1/ysaito/suite2p-pipeline-main/02_analysis/i166-m2_Exp01/01_ROI_detection_DeepCAD_all/suite2p/plane0/reg_tif', 'pth_dir': './pth', 'batch_size': 1, 'patch_t': 150, 'patch_x': 150, 'patch_y': 150, 'gap_y': 60, 'gap_x': 60, 'gap_t': 60, 'GPU': '0', 'ngpu': 1, 'num_workers': 0, 'scale_factor': 1, 'test_datasize': 8001, 'denoise_model': 'data_all_202305261842', 'visualize_images_per_epoch': False, 'save_test_images_per_epoch': True, 'colab_display': False, 'result_display': ''}


## Start testing
Here we lanuch the testing process. All results will be saved in the `/results` folder.

In [19]:
tc.run()

Stacks for processing -----> 
Total stack number ----->  8
file000_chan0.tif
file1000_chan0.tif
file2000_chan0.tif
file3000_chan0.tif
file4000_chan0.tif
file5000_chan0.tif
file6000_chan0.tif
file7000_chan0.tif
Using 1 GPU(s) for testing -----> 
[Model 1/1, E_20_Iter_3388.pth] [Stack 1/8, file000_chan0.tif] [Patch 17424/17424] [Time Cost: 1717 s] [ETA: 0 s]        
[Model 1/1, E_20_Iter_3388.pth] [Stack 2/8, file1000_chan0.tif] [Patch 17424/17424] [Time Cost: 1725 s] [ETA: 0 s]        
[Model 1/1, E_20_Iter_3388.pth] [Stack 3/8, file2000_chan0.tif] [Patch 17424/17424] [Time Cost: 1724 s] [ETA: 0 s]        
[Model 1/1, E_20_Iter_3388.pth] [Stack 4/8, file3000_chan0.tif] [Patch 17424/17424] [Time Cost: 1725 s] [ETA: 0 s]        
[Model 1/1, E_20_Iter_3388.pth] [Stack 5/8, file4000_chan0.tif] [Patch 17424/17424] [Time Cost: 1723 s] [ETA: 0 s]         
[Model 1/1, E_20_Iter_3388.pth] [Stack 6/8, file5000_chan0.tif] [Patch 17424/17424] [Time Cost: 1720 s] [ETA: 0 s]         
[Model 1/1, E_20